# Tarea 1 Comprobaciones previas (0,5 puntos)
 Muestra el número de documentos que se almacenan en la colección pedidos

In [13]:
import pymongo

# Establecer la cadena de conexión
conexion = "mongodb://127.0.0.1:27017"

# Intentar conectarse a MongoDB
try:
    cliente = pymongo.MongoClient(conexion)
    print("Conexión exitosa a MongoDB")
    # Seleccionar la base de datos
    db = cliente["supermercado"]
    # Contar el número de documentos en la colección 'pedidos'
    num_documentos = db["pedidos"].count_documents({})
    print("Número de documentos en la colección 'pedidos':", num_documentos)
except pymongo.errors.ConnectionFailure:
    print("Error al conectarse a MongoDB")

Conexión exitosa a MongoDB
Error al conectarse a MongoDB


# Tarea 2 Inserciones de registros (0,5 puntos)

### Inserta tres documentos en la colección utilizando un único comando:

- El primero tendrá como ID_Pedido el 79159 e Importe total 3.45
- El segundo tendrá el mismo ID_Pedido (79159), Importe total de 0.23 y Número de items 5.
- El tercero tendrá el mismo ID_Pedido (79159) y Número de items 8.

(Verifica que se han insertado los tres documentos)

In [2]:
# Insertar el primer documento
documento_1 = {
    "ID_Pedido": 79159,
    "Importe_total": 3.45
}

resultado = db["pedidos"].insert_one(documento_1)
print("ID del documento insertado:", resultado.inserted_id)


ID del documento insertado: 65e84e2a4911bdfab30a27a8


In [3]:
# Insertar el segundo documento
documento_2 = {
    "ID_Pedido": 79159,
    "Importe_total": 0.23,
    "Numero_de_items": 5
}

resultado = db["pedidos"].insert_one(documento_2)
print("ID del segundo documento insertado:", resultado.inserted_id)


ID del segundo documento insertado: 65e84e2a4911bdfab30a27a9


In [4]:
# Insertar el tercer documento
documento_3 = {
    "ID_Pedido": 79159,
    "Numero_de_items": 8
}

resultado = db["pedidos"].insert_one(documento_3)
print("ID del tercer documento insertado:", resultado.inserted_id)


ID del tercer documento insertado: 65e84e2a4911bdfab30a27aa


# Tarea 3 Consultas (3,5 puntos)

### 1. Dirección asociada al pedido 34789
Recupera la dirección asociada al pedido 34789. Solo nos interesa recuperar la dirección con todos sus campos

In [5]:
pedido_34789 = db["pedidos"].find_one({"ID_Pedido": 34789})

if pedido_34789:
    direccion_cliente_34789 = pedido_34789.get("Cliente", {}).get("Dirección", {})
    
    print("Datos de dirección del cliente del pedido 34789:")
    print(direccion_cliente_34789)
else:
    print("No se encontró el pedido con ID 34789")


No se encontró el pedido con ID 34789


### 2. Número de pedidos que NO disponen de dirección del cliente
Calcula el número de pedidos que no disponen de dirección del cliente.

In [6]:
num_pedidos_sin_direccion = db["pedidos"].count_documents({"Direccion": {"$exists": False}})
print("Número de pedidos sin dirección del cliente:", num_pedidos_sin_direccion)


Número de pedidos sin dirección del cliente: 3


### 3. Datos de cliente con el pedido de menor Importe total
Recupera los datos del cliente con el pedido de menor Importe total, incluyendo su ID_Pedido, Importe total y todos los datos del cliente con su dirección asociada.

In [7]:
import pymongo

pedido_menor_importe = db["pedidos"].find_one({}, sort=[("Importe total", 1)])

if pedido_menor_importe:
    numero_pedido_menor_importe = pedido_menor_importe["ID_Pedido"]
    print("Número de pedido con el menor importe total:", numero_pedido_menor_importe)

    pedido_completo_menor_importe = db["pedidos"].find_one({"ID_Pedido": numero_pedido_menor_importe})

    if pedido_completo_menor_importe:
        print("\nDatos completos del pedido con el menor importe total:")
        for key, value in pedido_completo_menor_importe.items():
            print(f"{key}: {value}")
    else:
        print("No se encontró ningún pedido con el número", numero_pedido_menor_importe)
else:
    print("No se encontraron pedidos en la base de datos.")


Número de pedido con el menor importe total: 79159

Datos completos del pedido con el menor importe total:
_id: 65e84e2a4911bdfab30a27a8
ID_Pedido: 79159
Importe_total: 3.45


### 4. Pedidos cuyo Importe total está entre 2 y 2.1 euros
Recupera los ID_Pedido de aquellos pedidos cuyo Importe total está entre 2 y 2.1 euros.

In [8]:
pedidos_rango_importe = db["pedidos"].find({"Importe total": {"$gte": 2, "$lte": 2.1}}, {"ID_Pedido": 1})
for pedido in pedidos_rango_importe:
    print("ID_Pedido:", pedido["ID_Pedido"])


### 5. Pedidos que incluyen Pan y Jabón entre sus ítems
Recupera los ID_Pedido de aquellos pedidos que incluyen Pan y Jabón entre sus ítems.

In [9]:
pedidos_pan_jabon = db["pedidos"].find({"Items": {"$all": ["Pan", "Jabón"]}}, {"ID_Pedido": 1})
for pedido in pedidos_pan_jabon:
    print("ID_Pedido:", pedido["ID_Pedido"])

# Tarea 4 Actualizaciones (2 puntos)

Te has dado cuenta de que al sumar el número de ítems en los pedidos, la persona encargada de recoger los datos tuvo un error, y dejó de incluir un elemento que todos tenían en común: la bolsa.

Debes realizar la siguiente actualización solo sobre aquellos pedidos que tienen Número de ítems:

- Incrementar en una unidad el valor actual de Número de items
- Añadir a la lista Items un nuevo documento:
  - Nombre: “Bolsa”
  - Importe unitario: 0.05
  - Cantidad: 1

Además de realizar la actualización, muestra el número de documentos que han sido actualizados.


In [10]:
import pymongo

# Establecer la cadena de conexión
conexion = "mongodb://172.18.0.2:27017"

# Intentar conectarse a MongoDB
try:
    cliente = pymongo.MongoClient(conexion)
    print("Conexión exitosa a MongoDB")
    # Seleccionar la base de datos
    db = cliente["supermercado"]
    # Incrementar en una unidad el valor actual de 'Número de items'
    result = db["pedidos"].update_many({}, {"$inc": {"Número de items": 1}})
    # Mostrar el número de documentos actualizados
    print("Número de documentos actualizados:", result.modified_count)
except pymongo.errors.ConnectionFailure:
    print("Error al conectarse a MongoDB")


Conexión exitosa a MongoDB
Número de documentos actualizados: 3


In [11]:
# Añadir un nuevo documento a la lista 'Items' en cada pedido
result = db["pedidos"].update_many({}, {"$push": {"Items": {"Nombre": "Bolsa", "Importe unitario": 0.05, "Cantidad": 1}}})

# Mostrar el número de documentos actualizados
print("Número de documentos actualizados:", result.modified_count)


Número de documentos actualizados: 3


# Tarea 5 Interacción con un Dataframe (2 puntos)
 Realiza las siguientes operaciones:
 Debes cargar todos los documentos existentes en la colección pedidos en un
 dataframe llamado df_pedidos.
 El dataframe debe utilizar como índice el campo ID_Pedido de la colección.
 A continuación, inserta en el Dataframe una nueva columna llamada Importe
 medio, que sea resultado de dividir el Importe total por el Número de ítems.
 Guarda el dataframe resultante, con la nueva columna, en una colección de la
 bbdd supermercado llamada resultadofinal.
 En la colección resultante no debe haber documentos con campos NaN

In [12]:
import pymongo
import pandas as pd

# Establecer la cadena de conexión
conexion = "mongodb://172.18.0.2:27017"

# Intentar conectarse a MongoDB
try:
    cliente = pymongo.MongoClient(conexion)
    print("Conexión exitosa a MongoDB")
    
    # Seleccionar la base de datos
    db = cliente["supermercado"]
    
    # Cargar todos los documentos de la colección 'pedidos' en un dataframe
    df_pedidos = pd.DataFrame(list(db["pedidos"].find()))

    # Establecer el campo 'ID_Pedido' como índice del dataframe
    df_pedidos.set_index('ID_Pedido', inplace=True)

    # Insertar una nueva columna 'Importe medio' en el dataframe
    df_pedidos['Importe medio'] = df_pedidos['Importe total'] / df_pedidos['Número de ítems']

    # Eliminar filas con valores NaN
    df_pedidos.dropna(inplace=True)

    # Guardar el dataframe resultante en la colección 'resultadofinal' de la base de datos 'supermercado'
    db["resultadofinal"].insert_many(df_pedidos.to_dict('records'))
    
except pymongo.errors.ConnectionFailure:
    print("Error al conectarse a MongoDB")


Conexión exitosa a MongoDB


KeyError: 'Importe total'

In [ ]:
# Imprimir el primer documento de la colección 'pedidos'
print(db["pedidos"].find_one())

